<figure>
    <center>
    <img src="../../figures/kinki2.png"  width="400" height="370" align="center"/>
    </center>
</figure>

# <span style="color:blue"><center>Hackathon: Shinkansen Travel Experience</center></span>

# <span style="color:red"><center>Process studied</center></span>

## <span style="color:green"><center>Notebook's Topic</center></span>

## <span style="color:blue">Authors</span>

1. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com
1. Juan Felipe Contreras Alcívar, j.contrerasa94@gmail.com

## <span style="color:blue">References</span>

1. [Ref to Link](link here)

## <span style="color:blue">Libraries</span>

In [154]:
import pandas as pd
import prince
import umap
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

## <span style="color:blue"> Importing Data </span>

In [31]:
train_df = pd.read_csv('../../input/train/imputed/train_imputed_df_hot_deck.csv', sep=';', index_col=0)
test_df = pd.read_csv('../../input/test/imputed/test_imputed_df_hot_deck.csv', sep=';', index_col=0)

df = pd.concat([train_df, test_df])

df

,Age,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,Gender,Customer_Type,Type_Travel,Travel_Class,Seat_Comfort,Seat_Class,...,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,Overall_Experience
ID,,,,,,,,,,,,,,,,,,,,,
98800001,52.0,272,0.0,5.0,Female,Loyal Customer,Personal Travel,Business,Needs Improvement,Green Car,...,Needs Improvement,Acceptable,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Good,Needs Improvement,Poor,0.0
98800002,48.0,2200,9.0,0.0,Male,Loyal Customer,Personal Travel,Eco,Poor,Ordinary,...,Poor,Good,Good,Excellent,Needs Improvement,Poor,Needs Improvement,Good,Good,0.0
98800003,43.0,1061,77.0,119.0,Female,Loyal Customer,Business Travel,Business,Needs Improvement,Green Car,...,Good,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent,1.0
98800004,44.0,780,13.0,18.0,Female,Loyal Customer,Business Travel,Business,Acceptable,Ordinary,...,Needs Improvement,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Acceptable,0.0
98800005,50.0,1981,0.0,0.0,Female,Loyal Customer,Business Travel,Business,Acceptable,Ordinary,...,Good,Excellent,Good,Good,Good,Good,Good,Good,Good,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935598,8.0,1334,0.0,0.0,Male,Loyal Customer,Personal Travel,Eco,Needs Improvement,Green Car,...,Needs Improvement,Acceptable,Acceptable,Good,Excellent,Good,Acceptable,Good,Acceptable,NaN
99935599,53.0,1772,0.0,0.0,Female,Loyal Customer,Business Travel,Business,Needs Improvement,Ordinary,...,Excellent,Excellent,Good,Good,Good,Good,Acceptable,Good,Good,NaN
99935600,22.0,1180,0.0,0.0,Male,Disloyal Customer,Business Travel,Eco,Good,Green Car,...,Good,Poor,Needs Improvement,Poor,Acceptable,Poor,Poor,Excellent,Needs Improvement,NaN


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129981 entries, 98800001 to 99935602
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Age                      129981 non-null  float64
 1   Travel_Distance          129981 non-null  int64  
 2   Departure_Delay_in_Mins  129981 non-null  float64
 3   Arrival_Delay_in_Mins    129981 non-null  float64
 4   Gender                   129981 non-null  object 
 5   Customer_Type            129981 non-null  object 
 6   Type_Travel              129981 non-null  object 
 7   Travel_Class             129981 non-null  object 
 8   Seat_Comfort             129981 non-null  object 
 9   Seat_Class               129981 non-null  object 
 10  Arrival_Time_Convenient  129981 non-null  object 
 11  Catering                 129981 non-null  object 
 12  Platform_Location        129981 non-null  object 
 13  Onboard_Wifi_Service     129981 non-null  object 
 14  

## <span style="color:blue"> Multiple Correspondence Analysis </span>

In [33]:
categorical_columns = [
    'Gender', 'Customer_Type', 'Type_Travel', 'Travel_Class', 'Seat_Comfort',
    'Seat_Class', 'Arrival_Time_Convenient', 'Catering', 'Platform_Location',
    'Onboard_Wifi_Service', 'Onboard_Entertainment', 'Online_Support',
    'Ease_of_Online_Booking', 'Onboard_Service', 'Legroom', 'Baggage_Handling',
    'CheckIn_Service', 'Cleanliness', 'Online_Boarding'
]

In [34]:
len(categorical_columns)

19

In [35]:
numerical_columns = ['Age', 'Travel_Distance', 'Departure_Delay_in_Mins', 'Arrival_Delay_in_Mins']

In [36]:
cat_df = df[categorical_columns]
num_df = df[numerical_columns]

In [37]:
cat_df.shape

(129981, 19)

In [38]:
num_df.shape

(129981, 4)

In [39]:
# Initialize MCA
mca = prince.MCA(
    n_components=60,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)

In [40]:
# Fit MCA on the data
mca = mca.fit(cat_df)

In [41]:
# Transform the data
mca_result = mca.transform(df)

In [42]:
mca_result

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
ID,,,,,,,,,,,,,,,,,,,,,
98800001,0.474800,-0.410125,0.312105,-0.204760,-0.163645,0.329465,0.121465,-0.756451,-0.230953,0.476512,...,-0.054320,-0.182547,-0.166228,-0.007134,-0.128215,-0.225288,0.227473,0.099770,0.305157,-0.184272
98800002,0.024769,0.335604,-0.129592,-0.498071,0.409629,-0.042740,0.034007,-0.142081,-0.122228,0.166583,...,0.463131,0.164597,0.197306,-0.063537,-0.034528,-0.181392,0.079372,0.142732,-0.055247,-0.167914
98800003,-0.554374,-0.291286,0.778519,-0.048927,-0.273788,-0.387368,0.161661,0.520966,-0.340431,0.058707,...,-0.010965,-0.051243,-0.031178,0.009635,0.050279,-0.040064,-0.051597,0.007842,0.047865,0.023667
98800004,0.489988,-0.318060,-0.319682,0.905683,-0.036013,-0.357114,0.263730,0.130189,-0.567747,0.150483,...,-0.000057,-0.196965,0.067780,-0.104990,-0.471172,0.026707,-0.012257,-0.221282,0.168031,-0.289839
98800005,-0.230322,-0.171035,-0.611387,-0.049179,-0.133819,-0.111407,0.076295,-0.000236,0.369373,-0.806818,...,-0.004723,0.149520,0.077029,-0.014209,-0.002231,-0.020777,-0.045654,-0.107267,0.068750,-0.049487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935598,0.218553,-0.282445,-0.202961,0.345470,-0.168415,-0.210844,0.240051,-0.030608,-0.550854,0.037236,...,0.040608,-0.000233,0.121735,0.051842,-0.029426,-0.121287,0.060337,0.091009,-0.090739,0.118057
99935599,-0.347806,-0.242823,-0.421588,-0.377102,-0.230614,-0.216909,0.177468,0.221113,-0.515488,-0.148299,...,0.050340,0.116460,0.042897,-0.047798,0.077985,0.179851,-0.135021,0.259601,0.191561,-0.326162
99935600,0.632652,-0.047702,0.341422,-0.288173,-0.113838,0.344793,-0.387558,0.206508,0.230302,0.186413,...,-0.252716,0.291011,0.088969,0.056570,-0.253741,0.418813,0.230397,0.163906,-0.245839,-0.261533


In [43]:
mca.cumulative_percentage_of_variance_

array([ 5.6708086 ,  9.95049486, 14.07605524, 18.01625586, 21.52461666,
       24.98894033, 28.36168993, 31.685628  , 34.89574065, 37.97627539,
       40.94935114, 43.7777316 , 46.54304415, 49.12982106, 51.50801796,
       53.62404768, 55.64505361, 57.29868571, 58.82022771, 60.29794282,
       61.74451919, 63.14638782, 64.51224767, 65.87016706, 67.22556222,
       68.5693214 , 69.91177068, 71.24908421, 72.52390325, 73.66653899,
       74.7915283 , 75.86823406, 76.92828145, 77.95913475, 78.92561445,
       79.87753279, 80.76459338, 81.63374124, 82.48449894, 83.31776027,
       84.13741503, 84.93634068, 85.7096199 , 86.44898988, 87.15744352,
       87.8471843 , 88.51783597, 89.18393571, 89.82489204, 90.45238321,
       91.07407126, 91.68262327, 92.26010669, 92.82068794, 93.36556476,
       93.90539178, 94.43532111, 94.94011753, 95.42629444, 95.90941623])

## <span style="color:blue"> Multiple Correspondence Analysis </span>

In [158]:
scaler = StandardScaler()
num_df = scaler.fit_transform(num_df)

In [159]:
num_df

array([[ 0.83155532, -1.66428139, -0.38652759, -0.26276334],
       [ 0.56699971,  0.213103  , -0.15005607, -0.39299386],
       [ 0.23630519, -0.89599494,  1.63661764,  2.70649255],
       ...,
       [-1.15261179, -0.78011903, -0.38652759, -0.39299386],
       [ 1.82363888, -1.52016682,  0.21778852,  0.02374381],
       [-1.28488959, -0.29324549, -0.38652759, -0.39299386]])

In [172]:
pca = PCA(3)

In [173]:
pca.fit(num_df)

PCA(n_components=3)

In [174]:
pca_result = pca.transform(num_df)

In [175]:
num_vector = pd.DataFrame(pca_result)

In [183]:
num_vector

,0,1,2
0,-0.784079,1.648329,-0.583720
1,-0.373689,0.204289,0.577609
2,2.853631,1.263540,-0.744147
3,-0.196043,1.009861,-0.634106
4,-0.582250,0.416808,0.529230
...,...,...,...
129976,-0.530889,-1.162785,-1.840467
129977,-0.628452,0.697102,0.519874
129978,-0.609616,-0.393687,-1.309291
129979,-0.196678,2.367295,0.151778


In [177]:
pca.explained_variance_ratio_

array([0.49714266, 0.30748457, 0.18546053])

## <span style="color:blue"> Join datasets </span>

In [203]:
red_df = mca_result.copy()

In [204]:
scaler = StandardScaler()

In [205]:
red_df = scaler.fit_transform(red_df)

In [206]:
red_df = pd.DataFrame(red_df)

In [207]:
red_df[60] = num_vector[0]
red_df[61] = num_vector[1]
red_df[62] = num_vector[2]

In [208]:
red_df.index = df.index

In [209]:
red_df

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
ID,,,,,,,,,,,,,,,,,,,,,
98800001,1.010297,-1.004550,0.778610,-0.522694,-0.442702,0.896935,0.335134,-2.102400,-0.653168,1.375695,...,-0.048255,-0.879943,-1.553621,1.582812,0.710624,2.216390,-1.340381,-0.784079,1.648329,-0.583720
98800002,0.052705,0.822020,-0.323294,-1.271433,1.108152,-0.116354,0.093829,-0.394886,-0.345677,0.480925,...,-0.429748,-0.236967,-1.250912,0.552291,1.016619,-0.401268,-1.221395,-0.373689,0.204289,0.577609
98800003,-1.179617,-0.713469,1.942176,-0.124896,-0.740667,-1.054571,0.446041,1.447918,-0.962787,0.169487,...,0.065168,0.345069,-0.276286,-0.359027,0.055855,0.347650,0.172154,2.853631,1.263540,-0.744147
98800004,1.042616,-0.779047,-0.797513,2.311948,-0.097425,-0.972207,0.727659,0.361834,-1.605669,0.434445,...,-0.710124,-3.233671,0.184177,-0.085291,-1.576103,1.220432,-2.108270,-0.196043,1.009861,-0.634106
98800005,-0.490087,-0.418929,-1.525231,-0.125540,-0.362014,-0.303295,0.210507,-0.000656,1.044640,-2.329289,...,-0.096105,-0.015314,-0.143284,-0.317675,-0.764017,0.499341,-0.359964,-0.582250,0.416808,0.529230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935598,0.465046,-0.691812,-0.506327,0.881886,-0.455607,-0.574001,0.662327,-0.085070,-1.557891,0.107501,...,0.350643,-0.201953,-0.836417,0.419837,0.648222,-0.659048,0.858741,-0.530889,-1.162785,-1.840467
99935599,-0.740075,-0.594763,-1.051739,-0.962631,-0.623870,-0.590514,0.489655,0.614537,-1.457871,-0.428141,...,-0.323289,0.535214,1.240280,-0.939508,1.849030,1.391328,-2.372478,-0.628452,0.697102,0.519874
99935600,1.346181,-0.116840,0.851749,-0.735623,-0.307960,0.938665,-1.069314,0.573947,0.651326,0.538176,...,0.382625,-1.741430,2.888203,1.603154,1.167436,-1.785561,-1.902371,-0.609616,-0.393687,-1.309291


In [210]:
red_train_df = red_df.loc[train_df.index]
red_test_df = red_df.loc[test_df.index]

In [211]:
red_train_df

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
ID,,,,,,,,,,,,,,,,,,,,,
98800001,1.010297,-1.004550,0.778610,-0.522694,-0.442702,0.896935,0.335134,-2.102400,-0.653168,1.375695,...,-0.048255,-0.879943,-1.553621,1.582812,0.710624,2.216390,-1.340381,-0.784079,1.648329,-0.583720
98800002,0.052705,0.822020,-0.323294,-1.271433,1.108152,-0.116354,0.093829,-0.394886,-0.345677,0.480925,...,-0.429748,-0.236967,-1.250912,0.552291,1.016619,-0.401268,-1.221395,-0.373689,0.204289,0.577609
98800003,-1.179617,-0.713469,1.942176,-0.124896,-0.740667,-1.054571,0.446041,1.447918,-0.962787,0.169487,...,0.065168,0.345069,-0.276286,-0.359027,0.055855,0.347650,0.172154,2.853631,1.263540,-0.744147
98800004,1.042616,-0.779047,-0.797513,2.311948,-0.097425,-0.972207,0.727659,0.361834,-1.605669,0.434445,...,-0.710124,-3.233671,0.184177,-0.085291,-1.576103,1.220432,-2.108270,-0.196043,1.009861,-0.634106
98800005,-0.490087,-0.418929,-1.525231,-0.125540,-0.362014,-0.303295,0.210507,-0.000656,1.044640,-2.329289,...,-0.096105,-0.015314,-0.143284,-0.317675,-0.764017,0.499341,-0.359964,-0.582250,0.416808,0.529230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98894375,1.154522,2.882434,-0.395549,-1.493658,-1.018321,-0.184153,0.103623,0.110986,0.710783,0.244233,...,-0.387942,0.371984,0.459190,1.130853,0.196931,1.287922,0.459712,3.162985,0.579777,-1.057854
98894376,0.092146,-0.400758,-0.794099,0.581700,-0.113592,0.245724,-0.561238,0.654382,0.547814,1.492357,...,-0.002864,0.029302,0.186004,-0.953266,-0.014849,0.824119,0.029736,-0.500372,1.089013,-0.742198
98894377,-0.520249,-0.763389,-1.073633,-1.212903,-0.741664,-0.895902,0.832011,0.635418,-2.114446,-0.673070,...,-0.156146,-0.199358,-0.131057,0.041132,0.449086,-0.169935,-0.030437,-0.496032,0.503665,1.691552


In [212]:
red_test_df

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
ID,,,,,,,,,,,,,,,,,,,,,
99900001,-0.762279,0.335063,1.028375,1.137984,-0.669574,-0.471639,0.087236,0.452678,1.603757,-1.244445,...,-0.146934,-0.183024,-0.934184,-0.845461,-1.494021,1.427482,-0.174281,-0.770670,0.699563,-1.123857
99900002,0.310619,0.578138,-0.188674,1.561222,1.504299,0.429866,-0.646765,0.662513,-0.983780,0.097693,...,0.019453,0.475127,-1.026380,0.094567,1.149401,0.220659,-0.408026,0.106646,-0.493353,-1.242610
99900003,-0.450732,-1.121794,1.476662,-0.241867,0.104928,1.263685,0.286644,-2.931500,-0.227397,0.704364,...,-0.327552,-0.182361,-0.169466,0.631125,-0.449176,-0.344235,0.124654,-0.478329,0.335577,1.581228
99900004,0.476233,1.630942,0.803154,0.914348,-1.597737,0.371565,0.250051,-1.080567,0.743149,-0.371700,...,0.710589,1.080999,-1.811338,-3.033735,-0.937437,0.687376,-0.900078,-0.607482,-0.172526,-0.869433
99900005,-1.370423,0.148250,1.690062,0.823639,-0.329070,0.436235,0.028013,-0.355086,-0.957968,-0.610436,...,1.424314,0.021221,-0.672795,0.461624,0.714619,0.037787,-0.685810,-0.212102,-0.816434,-1.216619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935598,0.465046,-0.691812,-0.506327,0.881886,-0.455607,-0.574001,0.662327,-0.085070,-1.557891,0.107501,...,0.350643,-0.201953,-0.836417,0.419837,0.648222,-0.659048,0.858741,-0.530889,-1.162785,-1.840467
99935599,-0.740075,-0.594763,-1.051739,-0.962631,-0.623870,-0.590514,0.489655,0.614537,-1.457871,-0.428141,...,-0.323289,0.535214,1.240280,-0.939508,1.849030,1.391328,-2.372478,-0.628452,0.697102,0.519874
99935600,1.346181,-0.116840,0.851749,-0.735623,-0.307960,0.938665,-1.069314,0.573947,0.651326,0.538176,...,0.382625,-1.741430,2.888203,1.603154,1.167436,-1.785561,-1.902371,-0.609616,-0.393687,-1.309291


In [213]:
red_train_df['Overall_Experience'] = train_df['Overall_Experience'].values

In [214]:
red_train_df.to_csv('../../input/train/red_train_df.csv', sep=';')
red_test_df.to_csv('../../input/test/red_test_df.csv', sep=';')

## <span style="color:blue"> PCA on complete matrix </span>

In [230]:
pca = PCA(60)

In [231]:
pca.fit(red_df)

PCA(n_components=60)

In [232]:
red_df_2 = pca.transform(red_df)

In [233]:
red_df_2

array([[-0.53895948,  1.50335413, -0.15752167, ..., -1.21558319,
         0.44314396, -0.4845246 ],
       [-0.38048873,  0.24243464,  0.09747684, ..., -0.20587221,
         0.93357209, -0.45089877],
       [ 2.70771284,  1.62812126, -0.21609482, ..., -0.16144718,
         0.11546228,  0.15573517],
       ...,
       [-0.69936171, -1.67381367, -2.13249409, ...,  1.39716554,
         1.72060126, -1.58188821],
       [-0.43935769,  2.24857384, -1.41406104, ..., -1.38363503,
         0.46737085,  1.76721331],
       [-0.50035962, -1.58359846, -1.80724063, ..., -0.18232481,
         1.23640576,  0.37950489]])

In [234]:
np.sum(pca.explained_variance_ratio_)

0.9636918151544059

In [236]:
red_df_2 = pd.DataFrame(red_df_2)
red_df_2.index = red_df.index

In [237]:
red_train_df_2 = red_df_2.loc[train_df.index]
red_test_df_2 = red_df_2.loc[test_df.index]

In [239]:
red_train_df_2['Overall_Experience'] = train_df['Overall_Experience'].values

In [240]:
red_train_df_2.to_csv('../../input/train/red_train_df_2.csv', sep=';')
red_test_df_2.to_csv('../../input/test/red_test_df_2.csv', sep=';')

## <span style="color:blue">Conclusions</span>

Conclusions of Notebook

## <span style="color:blue">Recomendations</span>

Recomendations of study made.